In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time


In [56]:
#Problem 1:

In [57]:
#loading diabetes data set
data = pd.read_csv("/content/drive/MyDrive/Dataset/diabetes_.csv")

In [58]:
#checking first rows of the dataset
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [59]:
#checking datatypes of the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [60]:
#check for missing values
missing_info = data.isnull().sum() /len(data) *100
print(missing_info)

Pregnancies                 0.0
Glucose                     0.0
BloodPressure               0.0
SkinThickness               0.0
Insulin                     0.0
BMI                         0.0
DiabetesPedigreeFunction    0.0
Age                         0.0
Outcome                     0.0
dtype: float64


In [61]:
#summary statistics od the datset columns
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [62]:
#since there are no missing values in the data set
# we wont be needing to handel the missing datas.

In [63]:
#seperate features X featire matrix X and variable Y
X = data.drop(columns=["Outcome"]).values #get all columnsexcept for outcoms
Y =data["Outcome"].values #get the outcome column only

In [64]:
#define a function for train-test split from scratch
def train_test_split_scratch(X,Y, test_size=0.3,random_seed=42):
  """
  Splits dataset into train and test sets.
  Arguments:
  X : np.ndarray
  Feature matrix.
  y : np.ndarray
  Target array.
  test_size : float
  Proportion of the dataset to include in the test split (0 < test_size < 1).
  random_seed : int
  Seed for reproducibility.
  Returns:
  X_train, X_test, y_train, y_test : np.ndarray
  Training and testing splits of features and target.
  """
  np.random.seed(random_seed) #ensures that the random number are not changed again and aagain
  indices = np.arange(X.shape[0]) #generaates random numbers that is the sma esize of the X

  #shuffle the data randomly to training and testing ste
  np.random.shuffle(indices)

  #Calculates the number of samples that should go into the test set.
  test_split_size = int(len(X)*test_size)

  #slices the shuffled datas as per the test_split_size calculated
  #and divide the data into training and testing set
  test_indices = indices[:test_split_size]
  train_indices = indices[test_split_size:]

  #select corresponding rows from X and Y useing the split indices
  #creates the final training and testing dataset
  X_train, X_test = X[train_indices], X[test_indices]
  Y_train, Y_test = Y[train_indices], Y[test_indices]
  return X_train,X_test,Y_train,Y_test

In [65]:
#store retun values from train_test_split_scratch function to get the final created 70% training data set an d30% tetsing dataset fomr X and Y
X_train, X_test, Y_train, Y_test = train_test_split_scratch(X,Y, test_size=0.3)


In [66]:
#ensuring that the all training and testing sets are of same dimension
print("Shape of X_train:",X_train.shape)
print("Shape of Y_train:",Y_train.shape)
print("Shape of X_test:",X_test.shape)
print("Shape of Y_test:",Y_test.shape)


Shape of X_train: (538, 8)
Shape of Y_train: (538,)
Shape of X_test: (230, 8)
Shape of Y_test: (230,)


In [67]:
def euclidean_distance(point1,point2):
  """
  Calculate the Euclidean distance between two points in n-dimensional space.
  Arguments:
  point1 : np.ndarray
  The first point as a numpy array.
  point2 : np.ndarray
  The second point as a numpy array.
  Returns:
  float
  The Euclidean distance between the two points.
  Raises:
  ValueError: If the input points do not have the same dimensionality.
  """
  #validation for unequal dimension for point1 and point2
  if point1.shape != point2.shape:
    raise ValueError("Point must have ethe same dimensions to calculate Euclidean distance.")
  # Calculate the Euclidean distance
  distance = np.sqrt(np.sum((point1-point2) ** 2))
  return distance


In [68]:
# try:
#   point1 = np.array([3,4])
#   point2 = np.array([0,0])
#   result = euclidean_distance(point1, point2)
#   expected_result = 5.0
#   assert np.isclose(result,expected_result),f"Expected {expected_result}, but got {result} "
#   print("test passed sucessfully")
# except ValueError as ve:
#   print(f"ValueError: {ve}")
# except  AssertionError as ae:
#   print(f"AssertionError: {ae}")
# except Exception as e:
#   print(f"Exception: {e}")


In [69]:
def knn_predict_single(query, X_train, Y_train, k=3):
  """
  Predict the class label for a single query using the K-nearest neighbors algorithm.
  Arguments:
  query : np.ndarray
  The query point for which the prediction is to be made.
  X_train : np.ndarray
  The training feature matrix.
  y_train : np.ndarray
  The training labels.
  k : int, optional
  The number of nearest neighbors to consider (default is 3).
  Returns:
  int
  The predicted class label for the query.
  """
  #Computes the Euclidean distance between the query point and every training sample
  distance = [euclidean_distance (query,x) for x in X_train]

  # sort the distances in ascending order.The closest points will appear first
  sorted_indices = np.argsort(distance)

  #Selects the indices of the k nearest neighbors. and get their labels
  nearest_indices=sorted_indices[:k]
  nearest_labels = Y_train[nearest_indices]

  # Counts how many times each label occurs among the nearest neighbors
  # and returns the label with the highest count
  prediction = np.bincount(nearest_labels).argmax()
  return prediction

In [70]:
def knn_predict(X_test, X_train, Y_train, k =3):
  """
  Predict the class labels for all test samples using the K-nearest neighbors algorithm.
  Arguments:
  X_test : np.ndarray
  The test feature matrix.
  X_train : np.ndarray
  The training feature matrix.
  y_train : np.ndarray
  The training labels.
  k : int, optional
  The number of nearest neighbors to consider (default is 3).
  Returns:
  np.ndarray
  An array of predicted class labels for the test samples.
  """
  prediction = [knn_predict_single(x,X_train,Y_train,k) for x in X_test]
  return np.array(prediction)

In [71]:
try:
# Define the test set for the test case
  X_test_sample = X_test[:5] # Taking a small subset for testing
  y_test_sample = Y_test[:5] # Corresponding labels for the subset
  # Make predictions
  predictions = knn_predict(X_test_sample, X_train, Y_train, k=3)
  # Print test results
  print("Predictions:", predictions)
  print("Actual labels:", y_test_sample)
  # Check if predictions match expected format
  assert predictions.shape == y_test_sample.shape, "The shape of predictions does not match the shape of the actual labels."
  print("Test case passed successfully!")
except AssertionError as ae:
  print(f"AssertionError: {ae}")
except Exception as e:
  print(f"An unexpected error occurred: {e}")


Predictions: [0 1 0 1 1]
Actual labels: [0 0 0 0 0]
Test case passed successfully!


In [72]:
def compute_accuracy(y_true, y_pred):
  """
  Compute the accuracy of predictions.
  Arguments:
  y_true : np.ndarray ; The true labels.
  y_pred : np.ndarray; The predicted labels.
  Returns:
  float : The accuracy as a percentage (0 to 100).
  """
  correct_predictions = np.sum(y_true == y_pred)
  total_predictions = len(y_true)
  accuracy = (correct_predictions / total_predictions) * 100
  return accuracy

In [73]:
try:
  # Make predictions on the entire test set
  predictions = knn_predict(X_test, X_train, Y_train, k=3)
  # Compute the accuracy
  accuracy = compute_accuracy(Y_test, predictions)
  # Print the accuracy
  print(f"Accuracy of the KNN model on the test set: {accuracy:.2f}%")
except Exception as e:
  print(f"An unexpected error occurred during prediction or accuracy computation: {e}")

Accuracy of the KNN model on the test set: 67.39%


In [74]:
#PRoblem 2

In [75]:
def scaledX(X):
    """
    Standardizes the feature matrix X.

    Parameters:
    X : np.ndarray
        Feature matrix of shape (n_samples, n_features)

    Returns:
    X_scaled : np.ndarray
        Standardized feature matrix with the same shape as X
    """
    return (X - X.mean(axis=0)) / X.std(axis=0)


In [76]:
X_scaled = scaledX(X)
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split_scratch(X_scaled, Y)


In [77]:
print("Shape of X_train:",X_train_s.shape)
print("Shape of Y_train:",y_train_s.shape)
print("Shape of X_test:",X_test_s.shape)
print("Shape of Y_test:",y_test_s.shape)

Shape of X_train: (538, 8)
Shape of Y_train: (538,)
Shape of X_test: (230, 8)
Shape of Y_test: (230,)


In [78]:
try:
# Define the test set for the test case
  X_test_sample = X_test_s[:5] # Taking a small subset for testing
  y_test_sample = y_test_s[:5] # Corresponding labels for the subset
  # Make predictions
  predictions = knn_predict(X_test_sample, X_train_s, y_train_s, k=3)
  # Print test results
  print("Predictions:", predictions)
  print("Actual labels:", y_test_sample)
  # Check if predictions match expected format
  assert predictions.shape == y_test_sample.shape, "The shape of predictions does not match the shape of the actual labels."
  print("Test case passed successfully!")
except AssertionError as ae:
  print(f"AssertionError: {ae}")
except Exception as e:
  print(f"An unexpected error occurred: {e}")


Predictions: [0 0 0 0 1]
Actual labels: [0 0 0 0 0]
Test case passed successfully!


In [79]:
try:
  # Make predictions on the entire scaled test set
  predictions = knn_predict(X_test_s, X_train_s, y_train_s, k=3)
  # Compute the accuracy
  accuracy = compute_accuracy(y_test_s, predictions)
  # Print the accuracy
  print(f"Accuracy of the KNN model on the test set: {accuracy:.2f}%")
except Exception as e:
  print(f"An unexpected error occurred during prediction or accuracy computation: {e}")

Accuracy of the KNN model on the test set: 70.87%


In [80]:
# Feature scaling improved kNN performance because the algorithm relies on
# distance calculations.
#If one feature has much bigger numbers than others, it will count more in the distance calculation.
# Example: Glucose = 200, Age = 50 → difference in Glucose matters 4× more than difference in Age.
# This means KNN mostly “looks at” the big-number feature and ignores smaller-number features.


In [ ]:
k_values = range(1, 16)
acc_original, acc_scaled = [], []
time_original, time_scaled = [], []

for k in k_values:
    start = time.time()
    y_pred = knn_predict(X_test, X_train, Y_train, k)
    time_original.append(time.time() - start)
    acc_original.append(compute_accuracy(y_test_s, y_pred))

    start = time.time()
    y_pred_s = knn_predict(X_test_s, X_train_s, y_train_s, k)
    time_scaled.append(time.time() - start)
    acc_scaled.append(compute_accuracy(y_test_s, y_pred_s))


In [ ]:
plt.plot(k_values, acc_original, label="Original Data")
plt.plot(k_values, acc_scaled, label="Scaled Data")
plt.xlabel("k")
plt.ylabel("Accuracy")
plt.legend()
plt.title("k vs Accuracy")
plt.show()


In [ ]:
plt.plot(k_values, time_original, label="Original Data")
plt.plot(k_values, time_scaled, label="Scaled Data")
plt.xlabel("k")
plt.ylabel("Time Taken (s)")
plt.legend()
plt.title("k vs Time Taken")
plt.show()
